In [3]:

import pandas as pd
import matplotlib.pyplot as plt
from v2_utils.generate_counterfactual import generate_CFs
from v2_utils.visualize_dataframe import Instances
from v2_utils.feature_importance import feature_importance_calculation

# Mock Data for Testing

In [ ]:
# We generate mock data for showing the process of generating counterfactuals, including mock_prototype, mock_data_outcome_0
# mock_prototype is the selected prototype data points
# mock_data_outcome_0 is the selected data points with the outcome 0 (which is expired), we use this data as query instance
# to find counterfactual examples of outcome as 2 (which is survived)

In [4]:
mock_prototype=pd.read_csv('./mock_data/mock_prototype.csv')
mock_data_outcome_0=pd.read_csv('./mock_data/mock_data_outcome_0.csv')


# Counterfactual Explanation

In [5]:
#Generate counterfactuals for the query instance
allcols=['lon', 'lat', 'code_cpr_type', 'code_etiology', 'code_gender',
       'code_race', 'code_location_type', 'code_acuity', 'code_witness',
       'code_cpr_used', 'code_who_cpr', 'code_aed_used', 'code_age',
       'code_duration']

features_not_vary=['code_gender','code_race']

continuous_feature_names=['lon','lat']
categorical_feature_names=['code_cpr_type', 'code_etiology', 'code_gender',
       'code_race', 'code_location_type', 'code_acuity', 'code_witness',
       'code_cpr_used', 'code_who_cpr', 'code_aed_used',  'code_duration','code_age']


# instance level

In [6]:
# Generate the counterfactuals for query instances with outcome== 0 
# get ith instance
i=10
query_instance=mock_data_outcome_0.iloc[i:i+1,:]
# generate counterfactuals
generate_cfs=generate_CFs(query_instance=query_instance, num_cfs=5, df_prototypes=mock_prototype, 
                          features_not_vary=features_not_vary, 
                          continuous_feature_names=continuous_feature_names,
                          categorical_feature_names=categorical_feature_names)
object_cf=generate_cfs.generate_counterfactuals()
# # visualize counterfactuals
instances=Instances(query_instance=object_cf.query_instance, df_cf=object_cf.df_cf, local_importance=None, 
                          local_impor_continus_pos=None,
                          local_impor_continus_neg=None,
                          local_impor_categorical_class=None)
instances.display()

Query instance (original outcome : 0)


,lat,lon,code_cpr_type,code_etiology,code_gender,code_race,code_location_type,code_acuity,code_witness,code_cpr_used,code_who_cpr,code_aed_used,code_age,code_duration,outcome
10,31.637223,-82.028627,8.0,7.0,0.0,2.0,8.0,1.0,2.0,0.0,0.0,0.0,3.0,1.0,0.0


Diverse Counterfactual set (new outcome : 1)


,lat,lon,code_cpr_type,code_etiology,code_gender,code_race,code_location_type,code_acuity,code_witness,code_cpr_used,code_who_cpr,code_aed_used,code_age,code_duration,outcome
20,31.75213673620241,-82.78367226231661,4.0,2.0,-,-,7.0,0.0,-,-,1.0,-,0.0,-,2.0
95,33.034638354934174,-82.61559788207353,10.0,3.0,-,-,13.0,2.0,1.0,1.0,1.0,1.0,0.0,-,2.0


In [7]:
# instance level- local feature importance
# local importance
importance=feature_importance_calculation(data_all=mock_data_outcome_0,global_importance=False,lst_object_cfs=None,
                            local_importance=True, object_cf=object_cf, 
                            allcols=allcols, continuous_feature_names=continuous_feature_names, 
                            categorical_feature_names=categorical_feature_names, features_not_vary=features_not_vary)
local_import=importance.cf_importance()

In [8]:
local_import

{'summary_importance:': {'lon': 1.0,
  'lat': 0.5,
  'code_cpr_type': 1.0,
  'code_etiology': 1.0,
  'code_gender': 0.0,
  'code_race': 0.0,
  'code_location_type': 1.0,
  'code_acuity': 1.0,
  'code_witness': 0.5,
  'code_cpr_used': 0.5,
  'code_who_cpr': 1.0,
  'code_aed_used': 0.5,
  'code_age': 1.0,
  'code_duration': 0.0},
 'summary_importance_continuous_positive': {'lon': 0.0, 'lat': 0.5},
 'summary_importance_continuous_negative': {'lon': 1.0, 'lat': 0.0},
 'summary_impor_categorical_class': {'code_cpr_type_class_9': 0.0,
  'code_cpr_type_class_1': 0.0,
  'code_cpr_type_class_8': 0.0,
  'code_cpr_type_class_4': 0.5,
  'code_cpr_type_class_3': 0.0,
  'code_cpr_type_class_7': 0.0,
  'code_cpr_type_class_14': 0.0,
  'code_cpr_type_class_11': 0.0,
  'code_cpr_type_class_2': 0.0,
  'code_cpr_type_class_6': 0.0,
  'code_cpr_type_class_5': 0.0,
  'code_cpr_type_class_13': 0.0,
  'code_cpr_type_class_10': 0.5,
  'code_cpr_type_class_12': 0.0,
  'code_etiology_class_7': 0.0,
  'code_etio

# total level

In [10]:
# when featurs_not_vary was set.
generate_cfs_total=generate_CFs(query_instance=mock_data_outcome_0, num_cfs=5, df_prototypes=mock_prototype,
                                features_not_vary=features_not_vary, 
                                continuous_feature_names=continuous_feature_names,
                                categorical_feature_names=categorical_feature_names)
lst_object_cfs=generate_cfs_total.generate_all_counterfactuals()


In [11]:
importance=feature_importance_calculation(data_all=mock_data_outcome_0,global_importance=True, lst_object_cfs=lst_object_cfs,
                            local_importance=False, object_cf=None, 
                            allcols=allcols, continuous_feature_names=continuous_feature_names, 
                            categorical_feature_names=categorical_feature_names, features_not_vary=features_not_vary)
global_importance=importance.cf_importance()


In [12]:
lst_object_cfs[10].query_instance

,10
lat,31.637223
lon,-82.028627
code_cpr_type,8.000000
code_etiology,7.000000
code_gender,0.000000
code_race,2.000000
code_location_type,8.000000
code_acuity,1.000000
code_witness,2.000000
code_cpr_used,0.000000


In [13]:
lst_object_cfs[10].df_cf

,lat,lon,code_cpr_type,code_etiology,code_gender,code_race,code_location_type,code_acuity,code_witness,code_cpr_used,code_who_cpr,code_aed_used,code_age,code_duration,outcome
20,31.752137,-82.783672,4.0,2.0,0.0,2.0,7.0,0.0,2.0,0.0,1.0,0.0,0.0,1.0,2.0
95,33.034638,-82.615598,10.0,3.0,0.0,2.0,13.0,2.0,1.0,1.0,1.0,1.0,0.0,1.0,2.0


In [14]:
global_importance

{'summary_importance:': {'lon': 0.8690476190476191,
  'lat': 0.8571428571428571,
  'code_cpr_type': 0.9404761904761905,
  'code_etiology': 0.8690476190476191,
  'code_gender': 0.0,
  'code_race': 0.0,
  'code_location_type': 0.8571428571428571,
  'code_acuity': 0.7023809523809523,
  'code_witness': 0.7321428571428571,
  'code_cpr_used': 0.43452380952380953,
  'code_who_cpr': 0.6964285714285714,
  'code_aed_used': 0.5297619047619048,
  'code_age': 0.7023809523809523,
  'code_duration': 0.7440476190476191},
 'summary_importance_continuous_positive': {'lon': 0.49404761904761907,
  'lat': 0.42857142857142855},
 'summary_importance_continuous_negative': {'lon': 0.375,
  'lat': 0.42857142857142855},
 'summary_impor_categorical_class': {'code_cpr_type_class_9': 0.07738095238095238,
  'code_cpr_type_class_1': 0.07738095238095238,
  'code_cpr_type_class_8': 0.0,
  'code_cpr_type_class_4': 0.08928571428571429,
  'code_cpr_type_class_3': 0.09523809523809523,
  'code_cpr_type_class_7': 0.083333333